<h1>Exercise 32</h1>
<p>Event count on/off, and simplify the logs to only register on and off once per consecutive occurence (count "on" after "off" as a new log).</p>
<strong>Expected outcome:</strong>
<br>
<img src="Day-32-Expected_Output.png" width="200">

In [5]:
import os
import sqlite3
import pandas as pd

db_name="exercise32.db"
conn=sqlite3.connect(db_name)
cursor=conn.cursor()

cursor.execute("""CREATE TABLE IF NOT EXISTS event_status(event_time VARCHAR(10), status VARCHAR(10))""")
conn.commit()

event_data=[
    ('10:01','on'),
    ('10:02','on'),
    ('10:03','on'),
    ('10:04','off'),
    ('10:07','on'),
    ('10:08','on'),
    ('10:09','off'),
    ('10:11','on'),
    ('10:12','off')
]
cursor.executemany("INSERT INTO event_status VALUES(?,?)",event_data)

querytoexec="SELECT * FROM event_status"
df=pd.read_sql(querytoexec,conn)
print(df)

  event_time status
0      10:01     on
1      10:02     on
2      10:03     on
3      10:04    off
4      10:07     on
5      10:08     on
6      10:09    off
7      10:11     on
8      10:12    off


In [6]:
querytoexec2 = "SELECT MIN(event_time) AS login, MAX(event_time) AS logout, COUNT(1) - 1 AS on_count FROM (SELECT *, SUM(CASE WHEN status = 'on' AND prev_status = 'off' THEN 1 ELSE 0 END) OVER (ORDER BY event_time) AS group_key FROM (SELECT *, LAG(status, 1, status) OVER (ORDER BY event_time ASC) AS prev_status FROM event_status) A) xxx GROUP BY group_key"
df=pd.read_sql(querytoexec2,conn)
print(df)
conn.close()

   login logout  on_count
0  10:01  10:04         3
1  10:07  10:09         2
2  10:11  10:12         1
